In [57]:
%run "1-MetricasDeEvaluacionDeIR.ipynb"
%run "2-BusquedaBinariaUsandoIndiceInvertido.ipynb"

In [58]:
import json
import numpy as np

import csv

## Punto 3

### Construcción matriz/vector tf-idf a partir del índice invertido

In [59]:
def crear_tf_idf_matrix(inverted_index: dict[dict[int]], 
                        tf_log_scale: bool = True,
                        normalize_matrix: bool = True) -> tuple[np.array, list[str], list[int]]:
    """
    Calcula la matriz TF-IDF a partir de un índice invertido.

    Esta función toma un índice invertido que mapea términos a documentos y sus frecuencias, 
    y calcula la matriz TF-IDF para cada término en cada documento. La opción de usar escala 
    logarítmica para la frecuencia de términos (TF) es configurable, así como la opción de 
    normalizar la matriz resultante.

    Args:
        inverted_index (dict[dict[int]]): 
            Un diccionario donde las claves son términos (str) y los valores son diccionarios.
            Estos diccionarios internos mapean el ID del documento (int) a la frecuencia de ese 
            término en el documento.

        tf_log_scale (bool, opcional): 
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. Si 
            se establece en False, se usará la frecuencia sin escala logarítmica.

        normalize_matrix (bool, opcional): 
            Indica si se debe normalizar la matriz TF-IDF resultante. La normalización se realiza por 
            filas, lo que significa que cada vector de documento se ajusta para que su norma sea 1. 
            Por defecto es True.

    Returns:
        tuple[np.array, list[str], list[int]]: 
            Retorna una tupla que contiene:
            - tf_idf_matrix (np.array): Un array bidimensional donde cada fila representa un documento 
              y cada columna representa un término. Los valores en la matriz son los pesos TF-IDF 
              correspondientes.
            - terms (list[str]): Una lista ordenada de los términos presentes en el índice invertido.
            - docs (list[int]): Una lista ordenada de los IDs de documentos únicos en el corpus.
    """

    # Obtiene y ordena los términos únicos en el índice invertido.
    terms = np.array(sorted(list(inverted_index.keys())))

    # Extrae y ordena los IDs de documentos únicos en el corpus.
    docs = {doc for docs_freq in inverted_index.values() for doc in docs_freq}
    docs = np.array(sorted(list(docs)))

    # Calcula el Inverse Document Frequency (IDF) para cada término.
    # IDF = log10(N / df), donde N es el número total de documentos y df es la frecuencia de documentos 
    # que contienen el término.
    idf = {term: np.log10(len(docs) / len(inverted_index[term]))
           for term in terms}

    # Calcula el Term Frequency (TF) para cada término en cada documento.
    # Si se aplica escala logarítmica, se usa la fórmula: TF = log10(1 + frecuencia).
    # Si no, se utiliza la frecuencia directa.
    if tf_log_scale:
        tf = {doc: {term: np.log10(1 + inverted_index[term].get(doc, 0)) 
                    for term in terms} for doc in docs}
    else:
        tf = {doc: {term: inverted_index[term].get(
            doc, 0) for term in terms} for doc in docs}

    # Calcula el TF-IDF para cada término en cada documento.
    # TF-IDF = TF * IDF para cada término en cada documento.
    tf_idf = {doc: {term: tf[doc][term] * idf[term]
                    for term in terms} for doc in docs}

    # Convierte el diccionario de TF-IDF en una matriz numpy para facilitar el procesamiento posterior.
    # Cada fila de la matriz representa un documento y cada columna un término.
    tf_idf_matrix = np.array([[tf_idf[doc][term] for term in terms] for doc in docs])
    
    # Normaliza la matriz TF-IDF, de manera que la norma de cada vector de documento sea 1.
    if normalize_matrix:
        tf_idf_matrix = tf_idf_matrix / np.linalg.norm(tf_idf_matrix, axis=1, keepdims=True)
    
    # Retorna la matriz TF-IDF, la lista de términos y la lista de documentos en el orden correspondiente.
    # Redondea a 7 decimales para evitar errores de precisión
    return tf_idf_matrix.round(7), terms, docs, idf


def crear_vector_tf_idf(text: str,
                        terms: list[str],
                        corpus_idf: dict[str, float],
                        tf_log_scale: bool = True,
                        normalize_vector: bool = True) -> np.array:
    """
    Crea un vector TF-IDF a partir de un texto dado.

    Esta función toma un texto, una lista de términos y un diccionario de valores 
    IDF precomputados para un corpus, y genera un vector TF-IDF para el texto 
    proporcionado. La opción de usar escala logarítmica para la frecuencia de términos 
    (TF) y la normalización del vector resultante es configurable.

    Args:
        text (str): 
            El texto a partir del cual se generará el vector TF-IDF.
        
        terms (list[str]): 
            Una lista de términos que se consideran en el corpus. Cada término de la lista 
            corresponde a una columna en el vector TF-IDF.

        corpus_idf (dict[str, float]): 
            Un diccionario que mapea cada término a su valor IDF (Inverse Document Frequency) 
            precomputado en el corpus. 

        tf_log_scale (bool, opcional): 
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. 
            Si se establece en False, se usará la frecuencia sin escala logarítmica.

        normalize_vector (bool, opcional): 
            Indica si se debe normalizar el vector TF-IDF resultante. La normalización ajusta 
            el vector para que su norma sea 1, lo que facilita la comparación entre textos.
            Por defecto es True.

    Returns:
        np.array: 
            Un array de NumPy que representa el vector TF-IDF del texto dado. Cada posición en el 
            vector corresponde a un término en la lista `terms`, y su valor es el peso TF-IDF 
            correspondiente a ese término en el texto.
    """    

    # Procesa el texto para limpiarlo y normalizarlo, preparando los datos para el análisis.
    # process_text fue definido en el notebook 2-BusquedaBinariaUsandoIndiceInvertido.ipynb
    text_cln = process_text({'0': text})['0']

    # Calcula el vector TF-IDF, aplicando escala logarítmica si se especifica.
    if tf_log_scale:
        txt_vector = np.array([
            np.log10(1 + text_cln.count(term)) * corpus_idf[term] if term in text_cln else 0
            for term in terms])
    else:
        # Si no se aplica escala logarítmica, se utiliza la frecuencia directa.
        txt_vector = np.array([
            text_cln.count(term) * corpus_idf[term] if term in text_cln else 0
            for term in terms])

    # Normaliza el vector TF-IDF, de manera que su norma sea 1 si la opción está activada.
    if normalize_vector:
        txt_vector = txt_vector / np.linalg.norm(txt_vector)
    
    # Redondea los valores del vector a 7 decimales para evitar errores de precisión.
    return txt_vector.round(7)

### Calculo de la similitud coseno

In [60]:
def cosine_simi(v1: np.array, v2: np.array, asume_norm_1: bool = False) -> float:
    """
    Calcula la similitud coseno entre dos vectores.

    Esta función calcula la similitud coseno entre dos vectores `v1` y `v2`. 
    La similitud coseno es una medida de la similitud entre dos vectores 
    en un espacio vectorial que mide el coseno del ángulo entre ellos. 
    Se utiliza comúnmente en análisis de texto y en tareas de recuperación de información.

    Args:
        v1 (np.array): 
            El primer vector (como un array de NumPy) con el que se calculará la similitud.
        
        v2 (np.array): 
            El segundo vector (como un array de NumPy) con el que se calculará la similitud.
        
        asume_norm_1 (bool, opcional): 
            Si se establece en True, la función asume que ambos vectores `v1` y `v2` 
            ya están normalizados (es decir, su norma es 1). Esto permite omitir el 
            cálculo de las normas, mejorando la eficiencia. Por defecto es False.

    Returns:
        float: 
            Un valor de tipo float que representa la similitud coseno entre los dos vectores.
            Un valor cercano a 1 indica que los vectores son muy similares (paralelos), 
            mientras que un valor cercano a 0 indica que son ortogonales (no relacionados).

    """
    
    if asume_norm_1:
        # Si se asume que los vectores ya están normalizados (norma = 1), 
        # la similitud coseno se reduce al producto punto entre ellos.
        res = np.dot(v1, v2)
    else:
        # Si los vectores no están normalizados, se calcula la similitud coseno 
        # como el producto punto dividido por el producto de las normas de los vectores.
        res = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    
    # Redondea a 4 decimales para evitar errores de precisión
    return res.round(4)

### Realizar consultas para obtener documentos relevantes

In [61]:
def get_most_relevant_docs(text: str, 
                           terms: np.array, 
                           docs: np.array, 
                           corpus_idf: dict[str, float], 
                           tf_idf_matrix: np.array, 
                           relevance_treshold: float = 0,
                           tf_log_scale: bool = True,
                           normalize_vector: bool = True
                           ) -> np.array:
    """
    Obtiene los documentos más relevantes en función de una consulta de texto.

    Esta función toma un texto de consulta, calcula su vector TF-IDF, y luego compara 
    este vector con una matriz TF-IDF de un corpus utilizando la similitud coseno. 
    Los documentos cuya similitud coseno con la consulta es mayor que un umbral 
    especificado se consideran relevantes y se retornan en orden de relevancia.

    Args:
        text (str): 
            El texto de la consulta para el cual se desean encontrar los documentos más relevantes.
        
        terms (np.array[str]): 
            Un array de términos relevantes en el corpus, utilizado para construir el vector TF-IDF 
            de la consulta.
        
        docs (np.array[str]): 
            Un array de IDs o nombres de documentos en el corpus, donde cada documento se corresponde 
            con una fila en la matriz `tf_idf_matrix`.
        
        corpus_idf (dict[str, float]): 
            Un diccionario que mapea cada término a su valor IDF (Inverse Document Frequency) 
            precomputado en el corpus.

        tf_idf_matrix (np.array[float]): 
            Una matriz TF-IDF donde cada fila representa un documento y cada columna representa un 
            término del corpus. Esta matriz se usa para comparar la consulta con los documentos existentes.

        relevance_treshold (float, opcional): 
            Un umbral de relevancia. Solo se retornarán los documentos cuya similitud coseno con 
            la consulta sea mayor que este valor. Por defecto es 0, lo que significa que se incluirán 
            todos los documentos con una similitud positiva.
            
        tf_log_scale (bool, opcional):
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. 
            Si se establece en False, se usará la frecuencia sin escala logarítmica.
        
        normalize_vector (bool, opcional):
            Indica si se debe normalizar el vector TF-IDF de la consulta. La normalización ajusta 
            el vector para que su norma sea 1, lo que facilita la comparación con los vectores 
            de documentos. Por defecto es True.

    Returns:
        np.array[str]: 
            Un array de IDs o nombres de documentos ordenados por relevancia, desde el más relevante 
            hasta el menos relevante, según la similitud coseno con la consulta.
    """

    # Crear el vector TF-IDF para la consulta de texto dada, utilizando el corpus y la lista de términos.
    v_query = crear_vector_tf_idf(text, terms, corpus_idf, 
                                  tf_log_scale=tf_log_scale, 
                                  normalize_vector=normalize_vector)

    # Calcular la similitud coseno entre el vector de la consulta y la matriz TF-IDF del corpus.
    # Se asume que los vectores en la matriz TF-IDF ya están normalizados.
    cosine_similarities = cosine_simi(v_query, tf_idf_matrix.T, asume_norm_1=True)

    # Identificar los índices de los documentos cuya similitud coseno es mayor que el umbral de relevancia.
    indices = np.where(cosine_similarities > relevance_treshold)[0]

    # Ordenar los índices de los documentos por similitud coseno en orden descendente.
    sorted_indices = indices[np.argsort(-cosine_similarities[indices])]

    # Obtener los documentos correspondientes a los índices ordenados por relevancia.
    sorted_docs = [(docs[i], cosine_similarities[i]) for i in sorted_indices]
    
    return sorted_docs

### Crear matriz tf-idf a partir del índice invertido

In [62]:
inverted_index = json.loads(open('./output/inverted_index.json').read())

In [63]:
tf_idf_matrix, terms, docs, corpus_idf = crear_tf_idf_matrix(inverted_index, 
                                                             tf_log_scale=True,
                                                             normalize_matrix=True
                                                             )

### Ejecutar consultas

In [64]:
with open('./output/processed_queries.json') as f:
    processed_queries = json.loads(f.read())

In [65]:
queries_res = {}
with open("./output/RRDV_metrics/RRDV-consultas_resultados.tsv", "w") as f:
    for q,t in processed_queries.items():
        
        rel_docs = get_most_relevant_docs(
                                ' '.join(t), 
                                terms, 
                                docs, 
                                corpus_idf, 
                                tf_idf_matrix, 
                                relevance_treshold=0,
                                tf_log_scale=True,
                                normalize_vector=True
                                )
        
        queries_res[q] = rel_docs
        rel_docs = ','.join([f'{d[0]}:{d[1]}' for d in rel_docs])
        
        f.write(f"{q}\t{rel_docs}\n")

### Evaluación

In [66]:
# Calcular metricas de evaluacion
file_path = 'data/relevance-judgments.tsv'

queries = {}

# Abrir el archivo de relevance-judgments
with open(file_path, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    
    for row in reader:
        # la primera columna es el id del query
        query_id = row[0]
        
        # Extraer el id de los documentos relevantes y su score
        document_scores = row[1].split(',')
        
        documents = {}
        
        for document_score in document_scores:
            # Obtener el id del documento y su puntaje
            document_id, score = document_score.split(':')
            
            score = int(score)
            documents[document_id] = score
        
        queries[query_id] = documents

In [67]:
# Construimos los vectores de relevancia binaria para cada query
relevances = {}

for query, documents in queries.items():
    binary_array = []
    for d in queries_res[query]:
        binary_array.append(1 if d[0] in documents else 0)
    relevances[query] = binary_array

#### P@M

In [68]:
file_path = 'output/RRDV_metrics/P@M.txt'

with open(file_path, 'w') as file:
    for query, documents in queries.items():
        precision = precision_at_k(relevances[query], len(documents))
        file.write(f"{query}: {precision}\n")

#### R@M

In [69]:
file_path = 'output/RRDV_metrics/R@M.txt'

with open(file_path, 'w') as file:
    for query, documents in queries.items():
        recall = recall_at_k(relevances[query], len(documents), len(documents))
        file.write(f"{query}: {recall}\n")

#### MAP

In [70]:
map = mean_average_precision(list(relevances.values()))
map

0.7402507734672369

#### NDCG@M

In [71]:
# Construimos los vectores de relevancia para cada query
relevances_ranked = {}

for query, documents in queries.items():
    binary_array = []
    for d in queries_res[query]:
        binary_array.append(documents.get(d[0],0))
    relevances_ranked[query] = binary_array

In [72]:
file_path = 'output/RRDV_metrics/NDCG@M.txt'

with open(file_path, 'w') as file:
    for query, documents in queries.items():
        ndcg = ndcg_at_k(relevances_ranked[query], len(documents))
        file.write(f"{query}: {ndcg}\n")